<a href="https://colab.research.google.com/github/congruent352/GAN/blob/main/Unsupervised_Medical_Image_Translation_Using_Cycle_MedGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Unsupervised Medical Image Translation Using Cycle-MedGan](https://arxiv.org/abs/1903.03374)**


This paper is an application of Cycle-Gan to Medical Images.


## **Main Task**

**Motivation / Model Problems**

CT scans are taken quite often but they provide a risk, 1 out of 2,000, of developing fatal cancer. PET scans, of course involve a risk, of causing alergic which is less harmful than cancer. In this paper, by translating PET images to CT images, the risk could be leveraged. In sum, the main objective is to transform PET images to CT images and especially the Cycle-GAN techniques will be deployed.

**Input**

- For traininig

We assume that PET and CT scans are available. **However**, they are not paired. For example, PET scan of Mike and CT scan of Marry are available, but CT scan of Mike might not be available.

- For model

PET scans will be taken as an input.

**Output**

From PET scans, we want to obtain CT images.

**Remark**

> Image-to-image translation is a field in computer vision and machine learning techniques have shown huge success in this field, for example [*pix2pix*](https://phillipi.github.io/pix2pix/). For supervised learning framwork, a paired dataset is required, which is often difficult to acquire in realistic medical scenarios. So, here with only inputs, Cycle-GAN will be employed as an **unsupervised learning**.

**Comparison to other papers?**
> **Question** Is there any previous papers attacking similar problems?
> 
> Yes, but previous methods are supervised learning.
>
> Or [*UNIT*](https://github.com/mingyuliutw/UNIT) and [*Cycle-GAN*](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix)are available, and we have the [*review paper*](https://arxiv.org/abs/1809.07294)

**Parameters**

> Is there any parameters to be determined?
>
> If there is, then is there any good way to determine the parameters?

## **Methodology**

This paper extends **Cycle-GAN** by introducing two loss functions analogous to the perceptual and style transfer losses.

### **Cycle-GAN**

Let $\mathcal{X}$ and $\mathcal{Y}$ be the sets of PET images and CT images, respectively. The ultimate objective is to build a function $G_1:\mathcal{X} \to \mathcal{Y}$. Then we have two important goals to achieve:
> 1. The output of $G_1$ should look like a CT image.
>
> To accomplish this goal, we introduce the discriminator $D_1:\mathcal{Y} \to \{0, 1\}$. Basically, it tries to label the output of $G_1$ as 0. By fooling this discriminator, $G_1$ will generate images that look like CT scans.
>
> But this is not enough. What if $G_1$ maps all the PET images to just one CT scans? So here we have the second property to obtain.
>
> 2. The structures of $\mathcal{X}$ has to be preserved after the action of $G_1$.
>
> For this purpose, we introduce the inverse mapping $G_2:\mathcal{Y} \to \mathcal{X}$ such that $$ G_2 G_1 \approx \mathcal{I}_{\mathcal{X}}.$$ By measuring the pixel-wise disparity between the original image and the output of $G_2G_1$, we can train the $G_2$. Also, as we did before, we build a discriminator $D_2:\mathcal{X}\to \{0,1\}$ to train $G_2$.

In sum, we have four different networks $G_1, G_2, D_1$ and $D_2$ and the relations could be described in the following way:
<center><img src="https://drive.google.com/uc?export=view&id=1asCwseX_EOKwRoIoMm6e5uun41evZQ11" width="400" height="300">



### **Loss functions from Cycle-GAN**

**Adversarial Losses**

Using discriminators, we have 
\begin{align*}
\log(D_1(\mathcal{y})) + \log \left(1 - D_1 (G_1(\mathcal{x})) \right)
\\ \log(D_2(\mathcal{x})) + \log \left(1 - D_2 (G_2(\mathcal{y})) \right)
\end{align*}

**Cycle-Consistency**

And using the relationship between $G_1$ and $G_2$, we have 
\begin{align*}
&||x - G_2(G_1(x))||_1
\\ &||y - G_1 (G_2(y))||_1
\end{align*}

---

### **Non-adversarial Cycle Losses**

It has been reported that the pixel-wise losses could fail to capture the perceptual aspect of human judgement on image quality. When used in translation tasks, they often lead to results which lack sharpness and fine-detailed structures. (or blurry images)

![](https://drive.google.com/uc?export=view&id=18LndQyh364RbLuCOf7yjz87I7_vQfdQj)

To circumvent this issue, **feature-based** loss functions were introduced. For example, the [*MedGAN*](https://github.com/mp2893/medgan) and [*perceptual adversarial paper*](https://arxiv.org/pdf/1706.09138.pdf) utililized a combinatino of **perceptual** and **style** transfer losses. These two losses, however, are computed with true labels, which are unavailable in our problem setting. So in this paper, the unsupervised version of two losses will be discussed. 

**Perceptual Loss**

To begin with, we study the supervised version first. If we want to train an image translator $F:\mathcal{X} \to \mathcal{Y}$ and we have a pretrained feature extractor $\mathcal{N}$ on $\mathcal{Y}$. We can train $F$ using the neural network $\mathcal{N}$ by comparing the features from preselected layers of $\mathcal{N}$. 

![](https://drive.google.com/uc?export=view&id=1SveoClT1ss8LU2JXpIvYwyPvjIPKVomJ)

The comparison between two features extracted from $i^{\text{th}}$ could be computed as $$ ||\mathcal{N}_i (y) - \mathcal{N}_i (\mathcal{F}(x))||_1$$ Then we consider the weight for each preselected layer. 

In unsupervised learning setting, we exploit the cycle-consistency. So, we compare $x$ with $G_2(G_1(x))$.  Likewise, we compare $y$ with $G_1(G_2(y))$. In sum, we count $$ \mathcal{L}_{\text{cPercep}} = \sum\limits_{i=0}^{L} \lambda_{\text{cP}, i} (||F_i (x) - F_i(G_2(G_1(x)))||_1 + ||F_i (y) - F_i(G_1(G_2(y)))||_1).$$



**Style Transfer Loss**

For each extracted feature, we compute the Gram matrix (or covariance matrix) to see the distribution of pixel values, and regularized by the spatial height, width and depth. To be more specific,
$$ Gr_{i}(x)_{m,n} = \frac{1}{h_1 w_i d_1} \sum\limits_{h=1}^{h_i}\sum\limits_{w = 1}^{w_i} F_i(x)_{h, w, m} F_i(x)_{h, w, n}$$

Then by comparing the Gram matrices using Frobenius norm as $$ \mathcal{L}_{\text{style}} = \sum\limits_{i=1}^L \lambda_{\text{style}, i} \frac{1}{4d_i^2} \left( ||Gr_i(x) - Gr_i(G_2(G_1(x)))||_F^2 + ||Gr_i(y) - Gr_i(G_1(G_2(y)))||_F^2 \right)$$ we can inject the style.

In sum, the perceptual and style transfer loss could be depicted in the following way:

![](https://drive.google.com/uc?export=view&id=1D6B-dkz9sSeuiryfduSy_7UsLM6GzC86)

### **Final Loss**

Finally, we can incorporate all the losses and the problem is equivalent to the min-max optimization $$ \min\limits_{G_1, G_2} \max\limits_{D_1, D_2} \mathcal{L}_{\text{adv}}(G_1, D_1) + \mathcal{L}_{\text{adv}} (G_2, D_2) + \lambda_{\text{cP}} \mathcal{L}_{\text{cPercep}} + \lambda_{\text{cyc}}\mathcal{L}_{\text{cyc}}(G_1, G_2) + \lambda_{\text{style}}\mathcal{L}_{\text{style}}(G_1, G_2).$$

### **User-defined setting?**

We need to determine
> 1. Which pretrained neural network to use as a feature extractor - In this paper, **[BiGAN](https://github.com/jeffdonahue/bigan)** was used.
>
> 2. Which layers to use for feature extraction - In this paper, all the layers were used.
>
> 3. What if we use correlation matrix instead of Gram matrix. - This paper employs Gram matrix, but I wonder whether the correlation matrix is applied.